In [2]:
!conda install -y numpy
!conda install -y matplotlib
!pip3 install torchvision

Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 4.10.1
  latest version: 4.10.3

Please update conda by running

    $ conda update -n base -c defaults conda



# All requested packages already installed.

Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 4.10.1
  latest version: 4.10.3

Please update conda by running

    $ conda update -n base -c defaults conda



# All requested packages already installed.

Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple


In [2]:
import torchvision
import torch
from torchvision import transforms
import torchvision.models as models
import matplotlib.pyplot as plt
import torch.nn as nn

batchsz = 32
lr = 1e-3
epochs = 10
#device = torch.device('cuda')
torch.manual_seed(1234)   #为了方便以后能复现同样结果
#viz = visdom.Visdom()

vgg16 = models.vgg16()
vgg16.fc = nn.Linear(1000, 4)
#resnet = ResNet18(2)
print(vgg16)

train_data_path = "./train/"
test_data_path = "./test/"

VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace=True)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace=True)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace=True)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace=True)
    (16): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1

In [3]:
import numpy as np
from torchvision import transforms

transforms = transforms.Compose([            #[1]
 transforms.Resize(256),                    #[2]
 transforms.CenterCrop(224),                #[3]
 transforms.ToTensor(),                     #[4]
 transforms.Normalize(                      #[5]
 mean=[0.485, 0.456, 0.406],                #[6]
 std=[0.229, 0.224, 0.225]                  #[7]
 )])

#from PIL import Image
#img = Image.open("train/dog/dog.1.jpg")

#img_t = transforms(img)
#batch_t = torch.unsqueeze(img_t, 0)

#resnet18.eval()
#out = resnet18(batch_t)
#print(out.shape)



#_, indices = torch.sort(out, descending=True)
#percentage = torch.nn.functional.softmax(out, dim=1)[0] * 100









In [4]:
train_data = torchvision.datasets.ImageFolder(root=train_data_path, transform=transforms)
test_data = torchvision.datasets.ImageFolder(root=test_data_path, transform=transforms)


In [5]:
EPOCH = 10               # train the training data n times, to save time, we just train 1 epoch
LR = 0.001 
batch_size = 20
train_loader=torch.utils.data.DataLoader(train_data,batch_size=batch_size)
test_loader=torch.utils.data.DataLoader(test_data,batch_size=batch_size)

In [6]:
if torch.cuda.is_available():
    device = torch.device("cuda")
else:
    device = torch.device("cpu")

print(device)


cpu


In [7]:
#!pip3 install tensorboard

In [8]:
import torch.nn as nn

cnn = vgg16
print(cnn)  # net architecture

optimizer = torch.optim.Adam(cnn.parameters(), lr=LR)   # optimize all cnn parameters
loss_func = nn.CrossEntropyLoss()                       # the target label is not one-hotted

def accuracy(output, target, topk=(1,)):
    """Computes the accuracy over the k top predictions for the specified values of k"""
    with torch.no_grad():
        maxk = max(topk)
        batch_size = target.size(0)

        _, pred = output.topk(maxk, 1, True, True)
        pred = pred.t()
        correct = pred.eq(target.view(1, -1).expand_as(pred))

        res = []
        for k in topk:
            correct_k = correct[:k].reshape(-1).float().sum(0, keepdim=True)
            res.append(correct_k.mul_(100.0 / batch_size))
        return res

def fit(model, train_loader, optimizer, loss_func):
    print('Training')
    model.train()
    train_running_loss = 0.0
    train_running_correct = 0
    total = 0
    
    for step, (b_x, b_y) in enumerate(train_loader):   # gives batch data, normalize x when iterate train_loader
        optimizer.zero_grad()           # clear gradients for this training step
        output = model(b_x)               # cnn output
        loss = loss_func(output, b_y)   # cross entropy loss
        train_running_loss += loss.item()
        _, predicted = output.max(1)
        total += b_y.size(0)
        train_running_correct += predicted.eq(b_y).sum().item()
        
        loss.backward()                 # backpropagation, compute gradients
        optimizer.step()                # apply gradients
        
        train_running_loss += loss.item()
     
    train_loss = train_running_loss / len(train_loader)
    train_accuracy = 100. * train_running_correct / total
    return train_loss, train_accuracy
    
train_loss, train_accuracy = [], []

# training and testing
for epoch in range(EPOCH):
    #print(f"Epoch {epoch+1} of {EPOCH}")
    train_epoch_loss, train_epoch_accuracy = fit(
        cnn, train_loader, optimizer, loss_func
    )

    train_loss.append(train_epoch_loss)
    train_accuracy.append(train_epoch_accuracy)
    
    
    print('Train Loss: %.3f | Accuracy: %.3f'%(train_epoch_loss,train_epoch_accuracy))

plt.plot(train_accuracy,'-o')
plt.xlabel('epoch')
plt.ylabel('accuracy')
plt.title('Train vs Valid Accuracy')
plt.show()

VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace=True)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace=True)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace=True)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace=True)
    (16): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1

/Users/wguo/opt/anaconda3/lib/python3.8/site-packages/torch/nn/functional.py:718: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  ../c10/core/TensorImpl.h:1156.)
  return torch.max_pool2d(input, kernel_size, stride, padding, dilation, ceil_mode)


KeyboardInterrupt: 

In [39]:
#cnn.eval()

from PIL import Image

img = Image.open('./test/cat/1.jpg')
img = transforms(img)
img = img.unsqueeze(0)

prediction = cnn(img)
prediction = prediction.argmax()
print(prediction)

tensor(1)


In [38]:
from PIL import Image

labels = ['dog','cat']
img = Image.open('./test/10.jpg')
img = transforms(img)
img = img.unsqueeze(0)
img = img.to(device)
prediction = model(img)
prediction = prediction.argmax()

print(labels[prediction])

dog
